# Overview
- [Creating Unnested Dataset](https://www.kaggle.com/naotaka1128/creating-unnested-dataset)
- ネストされているデータセットをネストせずに操作できるようにする

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

INPUT_DIR = "../input/mlb-player-digital-engagement-forecasting"

In [2]:
for dirname, _, filenames in os.walk(Path(INPUT_DIR)):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/mlb-player-digital-engagement-forecasting/teams.csv
../input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv
../input/mlb-player-digital-engagement-forecasting/.gitkeep
../input/mlb-player-digital-engagement-forecasting/players.csv
../input/mlb-player-digital-engagement-forecasting/train_updated.csv
../input/mlb-player-digital-engagement-forecasting/train.csv
../input/mlb-player-digital-engagement-forecasting/example_test.csv
../input/mlb-player-digital-engagement-forecasting/seasons.csv
../input/mlb-player-digital-engagement-forecasting/awards.csv
../input/mlb-player-digital-engagement-forecasting/mlb/competition.cpython-37m-x86_64-linux-gnu.so
../input/mlb-player-digital-engagement-forecasting/mlb/__init__.py


## utils

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float64)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## load data

In [4]:
files = ["seasons", "teams", "players", "awards", "example_sample_submission"]

for file in files:
    df = pd.read_csv(Path(INPUT_DIR) / f"{file}.csv")
    reduce_mem_usage(df).to_pickle(Path(INPUT_DIR) / f"{file}.pickle", protocol=4)

Mem. usage decreased to  0.00 Mb (3.3% reduction)
Mem. usage decreased to  0.00 Mb (16.6% reduction)
Mem. usage decreased to  0.17 Mb (10.4% reduction)
Mem. usage decreased to  0.52 Mb (14.3% reduction)
Mem. usage decreased to  0.14 Mb (50.0% reduction)


In [5]:
for file in ['example_test', 'train_updated']:
    # drop playerTwitterFollowers, teamTwitterFollowers from example_test
    df = pd.read_csv(Path(INPUT_DIR) / f"{file}.csv").dropna(axis=1,how='all')
    daily_data_nested_df_names = df.drop('date', axis = 1).columns.values.tolist()

    for df_name in daily_data_nested_df_names:
        date_nested_table = df[['date', df_name]]

        date_nested_table = (date_nested_table[
          ~pd.isna(date_nested_table[df_name])
          ].
          reset_index(drop = True)
          )

        daily_dfs_collection = []

        for date_index, date_row in date_nested_table.iterrows():
            daily_df = pd.read_json(date_row[df_name])

            daily_df['date'] = date_row['date']

            daily_dfs_collection = daily_dfs_collection + [daily_df]

        # Concatenate all daily dfs into single df for each row
        unnested_table = (pd.concat(daily_dfs_collection,
          ignore_index = True).
          # Set and reset index to move 'dailyDataDate' to front of df
          set_index('date').
          reset_index()
          )
        #print(f"{file}_{df_name}.pickle")
        #display(unnested_table.head(3))
        reduce_mem_usage(unnested_table).to_pickle(Path(INPUT_DIR) / f"{file}_{df_name}.pickle", protocol=4)
        #print('\n'*2)

        # Clean up tables and collection of daily data frames for this df
        del(date_nested_table, daily_dfs_collection, unnested_table)

Mem. usage decreased to  0.01 Mb (30.2% reduction)
Mem. usage decreased to  0.27 Mb (8.3% reduction)
Mem. usage decreased to  0.65 Mb (46.5% reduction)
Mem. usage decreased to  0.02 Mb (68.8% reduction)
Mem. usage decreased to  0.02 Mb (7.6% reduction)
Mem. usage decreased to  0.03 Mb (51.8% reduction)
Mem. usage decreased to  0.00 Mb (10.0% reduction)
Mem. usage decreased to  6.80 Mb (37.5% reduction)
Mem. usage decreased to 101.74 Mb (28.6% reduction)
Mem. usage decreased to  1.54 Mb (27.2% reduction)
Mem. usage decreased to 66.78 Mb (8.3% reduction)
Mem. usage decreased to 77.47 Mb (44.8% reduction)
Mem. usage decreased to  1.99 Mb (69.0% reduction)
Mem. usage decreased to  4.06 Mb (7.7% reduction)
Mem. usage decreased to  3.10 Mb (51.9% reduction)
Mem. usage decreased to  0.33 Mb (12.5% reduction)
Mem. usage decreased to 804.00 Mb (37.2% reduction)
Mem. usage decreased to  2.00 Mb (0.0% reduction)
Mem. usage decreased to  0.05 Mb (8.3% reduction)
